In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option('display.max_rows', 10)

**Step one - Read Excel file with Multiple Sheets

**Step two - Concatenate the sheets to one dataframe

**Step three - groupby employee number, using agg to get total payment and count of payments made by each employee

In [3]:
my_data = pd.read_excel("ippis_deduc_reports.xlsx", sheet_name = None)
my_data = pd.concat(my_data[frame] for frame in my_data.keys())
deduction = my_data.groupby(['Employee Number', 'Employee Name', 'Ministry Name']).agg(["sum","count"]).reset_index()

deduction.columns


MultiIndex([( 'Employee Number',      ''),
            (   'Employee Name',      ''),
            (   'Ministry Name',      ''),
            ('Deduction Amount',   'sum'),
            ('Deduction Amount', 'count'),
            (    'Element Name',   'sum'),
            (    'Element Name', 'count'),
            (          'Macro1',   'sum'),
            (          'Macro1', 'count'),
            (       'Auto_Open',   'sum'),
            (       'Auto_Open', 'count')],
           )

**Below, we changed Multi-Index to Index** This allows us drop (or choose) selected columns and rename thereafter.

In [4]:
deduction_columns = deduction.columns.map(''.join)
deduction.columns = deduction_columns

**Here, we selected the required columns to use and assigned it to the dataframe** After which we renamed the selected columns to prefered tags.

In [5]:
required_columns = ['Employee Number',
                    'Ministry Name',
                    'Employee Name',
                    'Deduction Amountsum',
                    'Deduction Amountcount',
                    ]

target_deduction = deduction[required_columns]
target_deduction.rename(columns = {'Deduction Amountsum': 'TotalRepayment', 'Deduction Amountcount': 'Nos Deducted'})


,Employee Number,Ministry Name,Employee Name,TotalRepayment,Nos Deducted
0,2.0,FEDERAL MINISTRY OF SCIENCE AND TECHNOLOGY,"BAMIKARERE, Mr. OLADELE ADEOLA",16524.62,2
1,5.0,NATIONAL POPULATION COMMISSION,"ARINZE, Ms. ONYINYE CHIKA",101045.32,2
2,65.0,FEDERAL MINISTRY OF MINES AND STEEL DEVELOPMENT,"IBRAHIM, Mr. ISYAKU",121032.57,3
3,211.0,POLICE SERVICE COMMISSION,"MOHAMMED, Mr. DAMARGU IDRIS",13363.66,2
4,615.0,MINISTRY OF DEFENCE - MOD,"YUSUF, Mrs. JIMADA FATIMA",18336.88,2
...,...,...,...,...,...
921,486387.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"OJO, Mr. SAMUEL OLUWATUYI",30722.16,3
922,486693.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"ADEDOKUN, Mr. SEUN TUNDE",10898.96,1
923,486762.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"OJO, Mr. KAYODE BABATUNDE",34785.76,1
924,486819.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"AYINDE, Mr. AYOMIDE OLATUNBOSUN",11103.12,3


In [106]:
cleaned_deduction = target_deduction.rename(columns = {'Deduction Amountsum': 'TotalRepayment', 'Deduction Amountcount': 'Nos Deducted'})

In [107]:
cleaned_deduction.to_excel('Cleaned_Deduction_Report.xlsx')

In [108]:
ippis_data = pd.read_csv("ippis_loandisk_2020_new.csv")
ippis_data.columns
# ippis_data.drop(columns = ['Remita Mandate Status',
#                             'Mobile','Principal After Deduct',
#                             'Account Number', 'Bank Code'
#                             ])


Index(['Released', 'Maturity', 'Loan#', 'Name', 'IPPIS No', ' Principal ',
       'Duration', 'Due ', 'Paid  ', 'Amortization  ', 'PendingDue  ',
       'Last Paid Date', 'Last Payment', 'Balance', 'MINISTRY/AGENCY',
       'Status'],
      dtype='object')

In [109]:
file_merge = pd.merge(
                      left = ippis_data,
                      right = cleaned_deduction,
                      how ="outer",
                      left_on ="IPPIS No",
                      right_on ="Employee Number"
)

In [121]:
file_merge = file_merge.replace({
                               'Duration' : '[A-Za-z]', 
                                },'', regex=True
)

In [122]:
file_merge.columns

Index(['Released', 'Maturity', 'Loan#', 'Name', 'IPPIS No', ' Principal ',
       'Duration', 'Due ', 'Paid  ', 'Amortization  ', 'PendingDue  ',
       'Last Paid Date', 'Last Payment', 'Balance', 'MINISTRY/AGENCY',
       'Status', 'Employee Number', 'Ministry Name', 'Employee Name',
       'TotalRepayment', 'Nos Deducted'],
      dtype='object')

In [123]:
new_merge = [
                'Released',
                'Maturity',
                'Loan#',
                'Name',
                'Employee Name',
                'IPPIS No',
                'Employee Number',    
                ' Principal ',
                'Amortization  ',
                'Duration',
                'Due ',
                'Paid  ',
                'TotalRepayment',
                'Nos Deducted',
                'Status',
                'PendingDue  ',
                'Balance',                
                'MINISTRY/AGENCY',
                'Ministry Name',
]
needed_merge = file_merge[new_merge]
needed_merge

,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,Duration,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name
0,07-12-20,07-03-21,CCM-3429654124,Okoye Ugochukwu Daniel,NaN,37503.0,NaN,18900.00,"8,347.43",3,25042.31,-,NaN,NaN,Processing,"25,042.31","25,042.31",Scientific Equipment Development Institute,NaN
1,27-10-20,27-01-21,CC-3429654123,"AMINU, HADIZA A","AMINU, Ms. HADIZA",127405.0,127405.0,49018.00,"21,649.46",3,64948.36,-,43298.90,2.0,Past Maturity,"64,948.36","64,948.36",NIGERIAN TELEVISION AUTHORITY,NIGERIAN TELEVISION AUTHORITY
2,22-10-20,22-01-21,CC-6706402307,"OKELOLA, KAYODE O","OKELOLA, Mr. KAYODE",250300.0,250300.0,71685.81,-,3,94976.53,"63,317.68",63317.68,2.0,Past Maturity,"31,658.85","31,658.85","FEDERAL MEDICAL CENTRE ABEOKUTA,",FEDERAL MEDICAL CENTRE ABEOKUTA
3,22-10-20,22-01-21,CC-645924972,"ALHASSAN, ALHASSAN LUKMAN A","ALHASSAN, Mr. ALHASSAN LUKMAN",464224.0,464224.0,63848.04,-,3,84592.26,"84,592.26",84592.26,3.0,Fully Paid,-,0,"NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,",NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES
4,22-10-20,22-01-21,CC-6733683418,"AYINLA, JANET OLUREMI A","AYINLA, Mrs. JANET OLUREMI",190057.0,190057.0,37118.30,-,3,49178.04,"16,392.68",16392.68,1.0,Past Maturity,"32,785.36","32,785.36","UNILORIN TEACHING HOSPITAL - ILORIN,",UNILORIN TEACHING HOSPITAL - ILORIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,17-09-20,17-12-20,CC-101222085,"AKOLO, JOSEPH PAUL A","AKOLO, Mr. JOSEPH PAUL",348836.0,348836.0,31235.40,-,3,38263.37,"38,263.38",38263.38,3.0,Fully Paid,-,-0.01,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA
1261,16-09-20,16-12-20,CC-9737170206,"MUHAMMAD, NAFIU M","MUHAMMAD, Mr. NAFIU",438725.0,438725.0,20306.66,-,3,24875.66,"16,583.78",16583.78,2.0,Past Maturity,"8,291.88","8,291.88",NaN,NIGERIAN POSTAL SERVICE
1262,16-09-20,16-12-20,CC-8805963535,"JIBRIN , OLABA IBRAHIM J","JIBRIN , Mr. OLABA IBRAHIM",307777.0,307777.0,9861.68,"4,026.84",3,12080.56,-,12080.55,3.0,Unsolicited Loan: resolved,"12,080.56","12,080.56",NaN,FEDERAL MINISTRY OF SCIENCE AND TECHNOLOGY
1263,15-09-20,15-12-20,CC-8815984159,"ABDULLAHI, HAMZA GAMBO A",NaN,168582.0,NaN,2256.00,921.2,3,2763.60,-,NaN,NaN,Past Maturity,"2,763.60","2,763.60",NaN,NaN


In [127]:
needed_merge.to_excel("Check.xlsx")

In [128]:
# needed_merge["Amortization"] = needed_merge["Amortization"].str.replace(',', '')

#  needed_merge["Amortization"] = needed_merge["Amortization"].astype(float)


# needed_merge["Due"] = needed_merge["Due"].apply(lambda x:(x.split()[0].replace(',', '')))

In [129]:
needed_merge = pd.read_excel("Check.xlsx")

In [130]:
needed_merge["Credit Difference"] = (needed_merge["Due "]).astype(float) - needed_merge["TotalRepayment"]

In [137]:
dateRel = pd.to_datetime(needed_merge["Released"])
needed_merge["Released"]= dateRel

MatDate = pd.to_datetime(needed_merge["Maturity"])
needed_merge["Maturity"]= MatDate

In [154]:
needed_merge['Paid  '] = needed_merge['Paid  '].str.replace('  -    ', '')

In [153]:
pd.options.display.float_format = '{:,.2f}'.format
needed_merge['Paid  '] = needed_merge['Paid  '].astype(float)

ValueError: could not convert string to float: 

In [138]:
needed_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Unnamed: 0         1265 non-null   int64         
 1   Released           1263 non-null   datetime64[ns]
 2   Maturity           1263 non-null   datetime64[ns]
 3   Loan#              1263 non-null   object        
 4   Name               1263 non-null   object        
 5   Employee Name      933 non-null    object        
 6   IPPIS No           1254 non-null   float64       
 7   Employee Number    933 non-null    float64       
 8    Principal         1264 non-null   float64       
 9   Amortization       1264 non-null   object        
 10  Duration           1263 non-null   float64       
 11  Due                1264 non-null   float64       
 12  Paid               1264 non-null   object        
 13  TotalRepayment     933 non-null    float64       
 14  Nos Dedu

In [132]:
not_deducted = needed_merge[needed_merge['TotalRepayment'].isnull()]
not_deducted

,Unnamed: 0,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,...,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name,Credit Difference
0,0,07-12-20,07-03-21,CCM-3429654124,Okoye Ugochukwu Daniel,NaN,37503.0,NaN,18900.00,"8,347.43",...,25042.31,-,NaN,NaN,Processing,"25,042.31","25,042.31",Scientific Equipment Development Institute,NaN,NaN
14,14,22-10-20,22-01-21,CCM-3612528438,"ISA, LAWAL I",NaN,365449.0,NaN,15500.00,"6,845.31",...,20535.95,-,NaN,NaN,Processing,"20,535.95","20,535.95","FEDERAL NEURO-PSYCHIATRIC HOSPITAL, KADUNA",NaN,NaN
15,15,22-10-20,22-01-21,CC-7979445573,"ITA, ANTIGHA ISRAEL I",NaN,34568.0,NaN,153932.88,"67,981.89",...,203945.67,-,NaN,NaN,Past Maturity,"203,945.67","203,945.67",SCIENTIFIC EQUIPMENT DEVEIOPMENT INSTITUTE(SED...,NaN,NaN
16,16,25-09-20,25-12-20,CCM-1763329616,ANTIGHA ISREAL / ITA,NaN,34568.0,NaN,56528.00,-,...,69246.80,"69,500.00",NaN,NaN,Fully Paid,-,-253.2,Scientific Equipment Deveiopment Institute (Se...,NaN,NaN
17,17,22-10-20,22-01-21,CC-793130927,"ADEGBOYEGA, ESTHER OLUFUNKE A",NaN,237174.0,NaN,169809.23,"74,993.41",...,224980.25,-,NaN,NaN,Past Maturity,"224,980.25","224,980.25","ENGINEERING MATERIALS DEVELOPMENT INSTITUTE,",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,1247,17-09-20,17-12-20,CC-2489858750,"ABBA, OJODOMO CALEB A",NaN,218713.0,NaN,38997.08,"15,923.80",...,47771.42,-,NaN,NaN,Past Maturity,"47,771.42","47,771.42","NATIONAL LOTTERY REGULATORY COMMISSION,",NaN,NaN
1248,1248,17-09-20,17-12-20,CC-331581236,"USMAN, KABIRU U",NaN,321053.0,NaN,69381.61,"28,330.83",...,84992.47,-,NaN,NaN,Past Maturity,"84,992.47","84,992.47","AHMADU BELLO UNIVERSITY TEACHING HOSPITAL,",NaN,NaN
1249,1249,17-09-20,17-12-20,CC-3985183946,"EMEASO, JONAH EZE E",NaN,220364.0,NaN,52803.33,"21,561.36",...,64684.08,-,NaN,NaN,Past Maturity,"64,684.08","64,684.08","NATIONAL LOTTERY REGULATORY COMMISSION,",NaN,NaN
1257,1257,17-09-20,17-12-20,CC-4958015097,"NWANYA, MAURICE NNAMDI N",NaN,205914.0,NaN,12953.47,"5,289.34",...,15868.00,-,NaN,NaN,Past Maturity,"15,868.00","15,868.00","NATIONAL LOTTERY REGULATORY COMMISSION,",NaN,NaN


In [133]:
deducted = needed_merge[needed_merge['TotalRepayment'].notnull()]
deducted

,Unnamed: 0,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,...,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name,Credit Difference
1,1,27-10-20,27-01-21,CC-3429654123,"AMINU, HADIZA A","AMINU, Ms. HADIZA",127405.0,127405.0,49018.00,"21,649.46",...,64948.36,-,43298.90,2.0,Past Maturity,"64,948.36","64,948.36",NIGERIAN TELEVISION AUTHORITY,NIGERIAN TELEVISION AUTHORITY,2.164946e+04
2,2,22-10-20,22-01-21,CC-6706402307,"OKELOLA, KAYODE O","OKELOLA, Mr. KAYODE",250300.0,250300.0,71685.81,-,...,94976.53,"63,317.68",63317.68,2.0,Past Maturity,"31,658.85","31,658.85","FEDERAL MEDICAL CENTRE ABEOKUTA,",FEDERAL MEDICAL CENTRE ABEOKUTA,3.165885e+04
3,3,22-10-20,22-01-21,CC-645924972,"ALHASSAN, ALHASSAN LUKMAN A","ALHASSAN, Mr. ALHASSAN LUKMAN",464224.0,464224.0,63848.04,-,...,84592.26,"84,592.26",84592.26,3.0,Fully Paid,-,0,"NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,",NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,-1.455192e-11
4,4,22-10-20,22-01-21,CC-6733683418,"AYINLA, JANET OLUREMI A","AYINLA, Mrs. JANET OLUREMI",190057.0,190057.0,37118.30,-,...,49178.04,"16,392.68",16392.68,1.0,Past Maturity,"32,785.36","32,785.36","UNILORIN TEACHING HOSPITAL - ILORIN,",UNILORIN TEACHING HOSPITAL - ILORIN,3.278536e+04
5,5,22-10-20,22-01-21,CC-69140781,"YAKUBU, ANDA YAHAYA Y","YAKUBU, Mr. ANDA YAHAYA",218313.0,218313.0,38101.62,-,...,50480.84,"50,480.85",50480.85,3.0,Fully Paid,-,-0.01,"NATIONAL LOTTERY REGULATORY COMMISSION,",NATIONAL LOTTERY REGULATORY COMMISSION,-1.000000e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,1259,17-09-20,17-12-20,CC-5768852538,"ABUBAKAR, LADAN A","ABUBAKAR, Mr. LADAN",156019.0,156019.0,28478.37,-,...,34886.00,"34,886.01",34886.01,3.0,Fully Paid,-,-0.01,"LEGAL AID COUNCIL OF NIGERIA,",LEGAL AID COUNCIL OF NIGERIA,-1.000000e-02
1260,1260,17-09-20,17-12-20,CC-101222085,"AKOLO, JOSEPH PAUL A","AKOLO, Mr. JOSEPH PAUL",348836.0,348836.0,31235.40,-,...,38263.37,"38,263.38",38263.38,3.0,Fully Paid,-,-0.01,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA,-1.000000e-02
1261,1261,16-09-20,16-12-20,CC-9737170206,"MUHAMMAD, NAFIU M","MUHAMMAD, Mr. NAFIU",438725.0,438725.0,20306.66,-,...,24875.66,"16,583.78",16583.78,2.0,Past Maturity,"8,291.88","8,291.88",NaN,NIGERIAN POSTAL SERVICE,8.291880e+03
1262,1262,16-09-20,16-12-20,CC-8805963535,"JIBRIN , OLABA IBRAHIM J","JIBRIN , Mr. OLABA IBRAHIM",307777.0,307777.0,9861.68,"4,026.84",...,12080.56,-,12080.55,3.0,Unsolicited Loan: resolved,"12,080.56","12,080.56",NaN,FEDERAL MINISTRY OF SCIENCE AND TECHNOLOGY,1.000000e-02


In [134]:
past_maturity = deducted[deducted.Status.isin(["Past Maturity", "Fully Paid"])]
past_maturity

,Unnamed: 0,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,...,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name,Credit Difference
1,1,27-10-20,27-01-21,CC-3429654123,"AMINU, HADIZA A","AMINU, Ms. HADIZA",127405.0,127405.0,49018.00,"21,649.46",...,64948.36,-,43298.90,2.0,Past Maturity,"64,948.36","64,948.36",NIGERIAN TELEVISION AUTHORITY,NIGERIAN TELEVISION AUTHORITY,2.164946e+04
2,2,22-10-20,22-01-21,CC-6706402307,"OKELOLA, KAYODE O","OKELOLA, Mr. KAYODE",250300.0,250300.0,71685.81,-,...,94976.53,"63,317.68",63317.68,2.0,Past Maturity,"31,658.85","31,658.85","FEDERAL MEDICAL CENTRE ABEOKUTA,",FEDERAL MEDICAL CENTRE ABEOKUTA,3.165885e+04
3,3,22-10-20,22-01-21,CC-645924972,"ALHASSAN, ALHASSAN LUKMAN A","ALHASSAN, Mr. ALHASSAN LUKMAN",464224.0,464224.0,63848.04,-,...,84592.26,"84,592.26",84592.26,3.0,Fully Paid,-,0,"NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,",NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,-1.455192e-11
4,4,22-10-20,22-01-21,CC-6733683418,"AYINLA, JANET OLUREMI A","AYINLA, Mrs. JANET OLUREMI",190057.0,190057.0,37118.30,-,...,49178.04,"16,392.68",16392.68,1.0,Past Maturity,"32,785.36","32,785.36","UNILORIN TEACHING HOSPITAL - ILORIN,",UNILORIN TEACHING HOSPITAL - ILORIN,3.278536e+04
5,5,22-10-20,22-01-21,CC-69140781,"YAKUBU, ANDA YAHAYA Y","YAKUBU, Mr. ANDA YAHAYA",218313.0,218313.0,38101.62,-,...,50480.84,"50,480.85",50480.85,3.0,Fully Paid,-,-0.01,"NATIONAL LOTTERY REGULATORY COMMISSION,",NATIONAL LOTTERY REGULATORY COMMISSION,-1.000000e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,1256,17-09-20,17-12-20,CC-8890591744,"AKOR, KINGSLEY A","AKOR, Mr. KINGSLEY",332336.0,332336.0,11391.07,-,...,13954.06,"13,954.05",13954.05,3.0,Past Maturity,0.01,0.01,UNIVERSITY OF PORTHARCOURT TEACHING HOSPITAL -...,UNIVERSITY OF PORTHARCOURT TEACHING HOSPITAL -...,1.000000e-02
1258,1258,17-09-20,17-10-20,CC-911097264,"YAHAYA, KAMAL ZAKARIYYA Y","YAHAYA, Mr. KAMAL ZAKARIYYA",345003.0,345003.0,19328.05,-,...,23676.86,"23,676.86",23676.86,1.0,Fully Paid,-,0,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA,0.000000e+00
1259,1259,17-09-20,17-12-20,CC-5768852538,"ABUBAKAR, LADAN A","ABUBAKAR, Mr. LADAN",156019.0,156019.0,28478.37,-,...,34886.00,"34,886.01",34886.01,3.0,Fully Paid,-,-0.01,"LEGAL AID COUNCIL OF NIGERIA,",LEGAL AID COUNCIL OF NIGERIA,-1.000000e-02
1260,1260,17-09-20,17-12-20,CC-101222085,"AKOLO, JOSEPH PAUL A","AKOLO, Mr. JOSEPH PAUL",348836.0,348836.0,31235.40,-,...,38263.37,"38,263.38",38263.38,3.0,Fully Paid,-,-0.01,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA,-1.000000e-02


In [ ]:
with pd.ExcelWriter('ippis_merged/ippis_merged.xlsx') as writer:
    not_deducted.to_excel(writer, sheet_name='Not_Deducted')
    not_on_loandisk.to_excel(writer, sheet_name='not_on_loandisk')
    past_maturity.to_excel(writer, sheet_name='past_maturity')
    deduction_schedule.to_excel(writer, sheet_name='deduction_schedule')